In [106]:
import sys
import pandas as pd
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lit
import math
assert sys.version_info >= (3, 8) # make sure we have Python 3.8+
from pyspark.sql import SparkSession, functions, types, Row
import re


line_re = re.compile(r"^(\S+) - - \[\S+ [+-]\d+\] \"[A-Z]+ \S+ HTTP/\d\.\d\" \d+ (\d+)$")


def line_to_row(line):
    """
    Take a logfile line and return a Row object with hostname and bytes transferred.
    Return None if regex doesn't match.
    """
    m = line_re.match(line)
    if m:
        # TODO
        return Row(hostname=m.group(1), bytes=m.group(2))
#         print((m.group(1)))
    else:
        return None


def not_none(row):
    """
    Is this None? Hint: .filter() with it.
    """
    return row is not None


def create_row_rdd(in_directory):
    log_lines = spark.sparkContext.textFile(in_directory + "/*")
    
    # TODO: return an RDD of Row() objects
    rows = log_lines.map(lambda x: line_to_row(x))
    rows = rows.filter(lambda x: not_none(x))
    return rows
    


def main(in_directory):
    logs = spark.createDataFrame(create_row_rdd(in_directory))
    

    # TODO: calculate r.

    r = 0 # TODO: it isn't zero.
    print(f"r = {r}\nr^2 = {r*r}")
    # Built-in function should get the same results.
    #print(totals.corr('count', 'bytes'))


# if __name__=='__main__':
#     in_directory = sys.argv[1]
#     spark = SparkSession.builder.appName('correlate logs').getOrCreate()
#     assert spark.version >= '3.2' # make sure we have Spark 3.2+
#     spark.sparkContext.setLogLevel('WARN')

#     main(in_directory)


In [3]:
in_directory = 'nasa-logs-1'
spark = SparkSession.builder.appName('correlate logs').getOrCreate()
assert spark.version >= '3.2' # make sure we have Spark 3.2+
spark.sparkContext.setLogLevel('WARN')



22/11/23 17:35:37 WARN Utils: Your hostname, Yvonnes-MacBook.local resolves to a loopback address: 127.0.0.1; using 192.168.0.26 instead (on interface en0)
22/11/23 17:35:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/23 17:35:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/23 17:35:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/23 17:35:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/23 17:35:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [94]:
logs = spark.createDataFrame(create_row_rdd(in_directory))
logs.show()

+--------------------+-----+
|            hostname|bytes|
+--------------------+-----+
|   in24.inetnebr.com| 1839|
|     uplherc.upl.com|    0|
|     uplherc.upl.com|    0|
|     uplherc.upl.com|    0|
|     uplherc.upl.com|    0|
|ix-esc-ca2-07.ix....| 1713|
|     uplherc.upl.com|    0|
|slppp6.intermind.net| 1687|
|piweba4y.prodigy.com|11853|
|slppp6.intermind.net| 9202|
|slppp6.intermind.net| 3635|
|ix-esc-ca2-07.ix....| 1173|
|slppp6.intermind.net| 3047|
|     uplherc.upl.com|    0|
|        133.43.96.45|10566|
|kgtyk4.kj.yamagat...| 7280|
|kgtyk4.kj.yamagat...| 5866|
|     d0ucr6.fnal.gov| 2743|
|ix-esc-ca2-07.ix....| 6849|
|     d0ucr6.fnal.gov|14897|
+--------------------+-----+
only showing top 20 rows



In [95]:
logs = logs.withColumn('bytes', logs['bytes'].cast(IntegerType()))
logs = logs.groupby('hostname').agg(functions.count('hostname').alias('requests'),
                                    functions.sum('bytes').alias('total_bytes'))
logs = logs.withColumn("requests^2", logs['requests']*logs['requests'])
logs = logs.withColumn("total_bytes^2", logs['total_bytes']*logs['total_bytes'])
logs = logs.withColumn("requests*total_bytes", logs['requests']*logs['total_bytes'])
logs = logs.withColumn("datapoints", lit(1))

In [96]:
logs.show()

+--------------------+--------+-----------+----------+--------------+--------------------+----------+
|            hostname|requests|total_bytes|requests^2| total_bytes^2|requests*total_bytes|datapoints|
+--------------------+--------+-----------+----------+--------------+--------------------+----------+
|ix-sea6-23.ix.net...|      18|     113562|       324|   12896327844|             2044116|         1|
|grimnet23.idirect...|      10|       7768|       100|      60341824|               77680|         1|
|   in24.inetnebr.com|      70|     674554|      4900|  455023098916|            47218780|         1|
|  pwestec.sierra.net|      16|      67462|       256|    4551121444|             1079392|         1|
|     pm6a3.sover.net|      22|      75874|       484|    5756863876|             1669228|         1|
|www-b5.proxy.aol.com|      46|     446840|      2116|  199665985600|            20554640|         1|
|ix-min1-02.ix.net...|      52|     232606|      2704|   54105551236|            1

In [98]:
sums = logs.groupBy().sum()

In [105]:
sums.first()


Row(sum(requests)=3944, sum(total_bytes)=72267472, sum(requests^2)=130240, sum(total_bytes^2)=102925029846104, sum(requests*total_bytes)=2648718932, sum(datapoints)=232)

In [107]:
n = sums.first()['sum(datapoints)']
x = sums.first()['sum(requests)']
y = sums.first()['sum(total_bytes)']
x2 = sums.first()['sum(requests^2)']
y2 = sums.first()['sum(total_bytes^2)']
xy = sums.first()['sum(requests*total_bytes)']

r = (n*xy-x*y)/(math.sqrt(n*x2-x*x)*math.sqrt(n*y2-y*y))
r

0.6300055382471266